<a href="https://colab.research.google.com/github/j0gea/Machine-Learning-Sidewalk-damage/blob/main/yolov5_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 구글 드라이브 파일 압축풀기

- s 는 추가파일이라 할 필요 x

In [ ]:
%cd /content/drive/MyDrive/yolov5/DATA/베리어프리존(장애물 없는 생활공간) 주행영상

/content/drive/MyDrive/yolov5/Data


In [ ]:
!ls

 1.동부산실외_베리어프리객체_1	 testdata     '[라벨]1.동부산실외_베리어프리객체_1_s.zip'
 custom.yaml			 train.cache  '[라벨]1.동부산실외_베리어프리객체_1.zip'
 images				 train.txt    '[원천]1.동부산실외_베리어프리객체_1.zip'
 labels				 valid.txt


In [ ]:
# 파일 압축풀기
%cd /content/drive/MyDrive/yolov5/Data/testdata/labels

!unzip -qq "/content/drive/MyDrive/yolov5/DATA/베리어프리존(장애물 없는 생활공간) 주행영상/[라벨]2.서부산실외_베리어프리객체_1.zip"

/content/drive/MyDrive/yolov5/Data/testdata/labels


In [ ]:
# 파일 압축풀기
%cd /content/drive/MyDrive/yolov5/Data/testdata/labels

!unzip -qq "/content/drive/MyDrive/yolov5/DATA/베리어프리존(장애물 없는 생활공간) 주행영상/[원천]2.서부산실외_베리어프리객체_1.zip"

In [ ]:
# 파일 삭제
# !rm -rf ./*.txt
# !rm -rf ./*.json

### YOLOv5 설치

yolov5 설정

In [ ]:
%cd 설치 폴더로 이동

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16802, done.
remote: Counting objects: 100% (343/343), done.
remote: Compressing objects: 100% (249/249), done.
remote: Total 16802 (delta 186), reused 179 (delta 94), pack-reused 16459
Receiving objects: 100% (16802/16802), 15.56 MiB | 12.62 MiB/s, done.
Resolving deltas: 100% (11485/11485), done.


In [ ]:
%cd yolov5

In [ ]:
! pip install -r requirements.txt

## YOLOv5 Colab
- https://velog.io/@jiiina/YOLO-Colab
- https://clap0107.tistory.com/5
- 저거써볼거임

- 생각해보니 JSON 파일도 함께 사용해야함
- https://velog.io/@1-june/YOLOv5%EC%97%90%EA%B2%8C-%EC%83%88%EB%A1%9C%EC%9A%B4-%EA%B2%83-%EA%B0%80%EB%A5%B4%EC%B3%90%EB%B3%B4%EA%B8%B0
- https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#local-logging

In [ ]:
# 런타임 > 런타임 유형 변경> 하드웨어 가속기에서 GPU를 선택
# GPU 제대로 잡히는지 확인

### YOLO 실행

#### 라벨 to txt

txt 변환
- 참고
- https://velog.io/@1-june/YOLOv5%EC%97%90%EA%B2%8C-%EC%83%88%EB%A1%9C%EC%9A%B4-%EA%B2%83-%EA%B0%80%EB%A5%B4%EC%B3%90%EB%B3%B4%EA%B8%B0
- https://minmiin.tistory.com/14

In [ ]:
import re
import json
import os
import csv

training_labels_folder = "/content/drive/MyDrive/yolov5/Data/testdata/labels"
training_images_folder = "/content/drive/MyDrive/yolov5/Data/testdata/images"
jsons = []
for f in os.listdir(training_images_folder):
  jsons.append(f[0:-3]+"json")

In [ ]:
def save_as_yolo_format(destination_folder, json_data):
    img_width = json_data['images']['width'] # JSON 파일에서 이미지의 width 값이 있는 곳
    img_height = json_data['images']['height'] # JSON 파일에서 이미지의 height 값이 있는 곳

    yolov5_format_list = []
    for i in json_data['annotations']: # 고려할 것들
        if i['category_id'] == 'flatness_D' or i['category_id'] == 'flatness_E' or i['category_id'] == 'block_state_broken':
          region_num = 0
        else:
          region_num = 1
        xmax = -1; xmin = 5000; ymax = -1; ymin = 5000
        for x, y in (i['segmentation']):
            if (x > xmax): xmax = x
            if (x < xmin): xmin = x
            if (y > ymax): ymax = y
            if (y < ymin): ymin = y

        xcentre = ((xmax + xmin) / 2)
        ycentre = ((ymax + ymin) / 2)
        bbox_width = (xmax - xmin)
        bbox_height = (ymax - ymin)
        yolov5_format = [region_num, xcentre, ycentre, bbox_width, bbox_height]
        yolov5_format_list.append(yolov5_format)

    file = open(os.path.join(destination_folder, re.sub(r'[^.]+$', 'txt', json_data['images']['file_name'])), 'w', newline='') # 파일명
    with file:
        write = csv.writer(file, delimiter= ' ')
        write.writerows(yolov5_format_list)

In [ ]:
destination_folder = "/content/drive/MyDrive/yolov5/Data/testdata/labels"

n=1
for j in jsons:
  with open(os.path.join(training_labels_folder, j)) as json_file:
    json_data = json.load(json_file)
    save_as_yolo_format(destination_folder, json_data)
    print(n , "/" , len(jsons))
    n=n+1

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/yolov5/Data/testdata/labels/DC0102_2020-11-12 141133_0P.json'

#### train / val 나누기

In [ ]:
# 먼저 export파일 안에 있는 이미지들을 list로 만들어준다.
from glob import glob
img_list = glob('/content/drive/MyDrive/yolov5/Data/images/train/*.jpg')
print(len(img_list))

7294


In [ ]:
# 다음에 img_list를 train set과 valid set으로 나눠준다.

from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size = 0.2, random_state = 2000)
print(len(train_img_list), len(val_img_list))

5835 1459


In [ ]:
file = open("/content/drive/MyDrive/yolov5/Data/train.txt", 'w', newline='') # 파일명
for i in train_img_list[0:50]:
    file.write(i+"\n")
#for i in train_label_list:
#    file.write(i+"\n")
file.close()

In [ ]:
file = open("/content/drive/MyDrive/yolov5/Data/valid.txt", 'w', newline='') # 파일명
for i in val_img_list[0:10]:
    file.write(i+"\n")
#for i in val_label_list:
#    file.write(i+"\n")
file.close()

#### yaml 파일 생성

In [ ]:
import yaml

yaml_data = {"names":['broken', 'not_broken'], # 클래스 이름
             "nc":2, # 클래스 수
             "path":"/content/drive/MyDrive/yolov5/Data", # root 경로
             "train":("train.txt"), # train.txt 경로
             "val":("valid.txt"), # valid.txt 경로
             # "test":os.path.join(root_dir,"test.txt") # test.txt 경로
             }

with open(("custom.yaml"), "w") as f:
  yaml.dump(yaml_data, f)

#### YOLOv5 실행
- 각 파라메터 뜻 https://ropiens.tistory.com/44
- https://github.com/ultralytics/yolov5/blob/master/data/coco128.yaml
- https://sguys99.github.io/ds01

In [ ]:
!python train.py --img 416 --batch 16 --epochs 50 --data /content/drive/MyDrive/yolov5/Data/custom.yaml --cfg ./models/yolov5m.yaml --weights yolov5m.pt --name results

2024-07-22 19:02:35.664923: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 19:02:35.664975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 19:02:35.666670: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=./models/yolov5m.yaml, data=/content/drive/MyDrive/yolov5/Data/custom.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=